<h1> AI Model that accurately predicts the outcome of any champion matchup in lane </h1>

<h2> Webscrape </h2>

<h4>Webscraping for list of all availible champions, respective winrates, pick rates, and ban rates. </h4>

In [117]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

response = requests.get('https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/champions/')
response.raise_for_status()
soup = BeautifulSoup(response.text, 'html.parser')
links = []
for a_tag in soup.find_all('a', href=True):
    href = a_tag['href']
    if href.endswith('.json') and not href =='-1.json':
        links.append(f"https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/champions/{href}")

def json_extract(obj, key):
    arr = []

    def extract(obj, arr, key):
        if isinstance(obj, dict):
            for k, v in obj.items():
                if k == key:
                    arr.append(v)
                elif isinstance(v, (dict,list)):
                    extract(v, arr, key)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)

        return arr
    
    values = extract(obj, arr, key)
    return values

champion_names = []
champion_ids = []

for url in links:
    champion_json = requests.get(url).json()
    
    id = json_extract(champion_json, 'id')
    name = json_extract(champion_json, 'name')
    champion_names.append(name[0])
    champion_ids.append(id[0])

championId_dict = dict(map(lambda i, j: (int(i), j), champion_ids, champion_names))

In [119]:
championId_dict #list of all champion names scraped from community dragon

{1: 'Annie',
 10: 'Kayle',
 101: 'Xerath',
 102: 'Shyvana',
 103: 'Ahri',
 104: 'Graves',
 105: 'Fizz',
 106: 'Volibear',
 107: 'Rengar',
 11: 'Master Yi',
 110: 'Varus',
 111: 'Nautilus',
 112: 'Viktor',
 113: 'Sejuani',
 114: 'Fiora',
 115: 'Ziggs',
 117: 'Lulu',
 119: 'Draven',
 12: 'Alistar',
 120: 'Hecarim',
 121: "Kha'Zix",
 122: 'Darius',
 126: 'Jayce',
 127: 'Lissandra',
 13: 'Ryze',
 131: 'Diana',
 133: 'Quinn',
 134: 'Syndra',
 136: 'Aurelion Sol',
 14: 'Sion',
 141: 'Kayn',
 142: 'Zoe',
 143: 'Zyra',
 145: "Kai'Sa",
 147: 'Seraphine',
 15: 'Sivir',
 150: 'Gnar',
 154: 'Zac',
 157: 'Yasuo',
 16: 'Soraka',
 161: "Vel'Koz",
 163: 'Taliyah',
 164: 'Camille',
 166: 'Akshan',
 17: 'Teemo',
 18: 'Tristana',
 19: 'Warwick',
 2: 'Olaf',
 20: 'Nunu & Willump',
 200: "Bel'Veth",
 201: 'Braum',
 202: 'Jhin',
 203: 'Kindred',
 21: 'Miss Fortune',
 22: 'Ashe',
 221: 'Zeri',
 222: 'Jinx',
 223: 'Tahm Kench',
 23: 'Tryndamere',
 233: 'Briar',
 234: 'Viego',
 235: 'Senna',
 236: 'Lucian',
 2

In [121]:
import re
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

response = requests.get('https://u.gg/lol/champions', headers=headers)
response.raise_for_status()  # Check if the request was successful
soup = BeautifulSoup(response.text, 'html.parser')
baseURL = "https://u.gg/"

champContainer = soup.find('div', class_='champion-home-page')
champion_data = []
if champContainer:
    links = champContainer.find_all('a')

    for link in links:
        url = baseURL + link['href']
        resp = requests.get(url, headers=headers)
        if resp.status_code == 200:
            html = resp.text
            soup1 = BeautifulSoup(html, 'html.parser')
            rankingStats = soup1.find('div', class_='champion-ranking-stats-normal')
            if rankingStats:
                text = rankingStats.text.strip()
                tier_pattern = r"([A-S]\+?Tier)"
                redTier_pattern = r"([A-S]\+)"
                win_rate_pattern = r"(\d+(\.\d+)?%)Win Rate"
                rank_pattern = r"(\d+ / \d+)Rank"
                pick_rate_pattern = r"(\d+\.\d+%)Pick Rate"
                ban_rate_pattern = r"(\d+\.\d+%)Ban Rate"
                matches_pattern = r"(\d+,?\d*,?\d*)Matches"
                
                # Apply patterns
                tier = re.search(tier_pattern, text).group(1)
                win_rate = re.search(win_rate_pattern, text).group(1)
                rank = re.search(rank_pattern, text).group(1)
                pick_rate = re.search(pick_rate_pattern, text).group(1)
                ban_rate = re.search(ban_rate_pattern, text).group(1)
                matches = re.search(matches_pattern, text).group(1)

                numbers = rank.split('/')

                # Strip whitespace and convert to integers
                numerator = int(numbers[0].strip())
                denominator = int(numbers[1].strip())

                # Perform division
                rank = f"{((numerator / denominator)*100):.2f}%"

                name = soup1.find('span', class_='champion-name').text.strip()
                if name == 'Red Kayn':
                    name = 'Kayn'

                champion_data.append({
                'name' : name,
                'tier': tier,
                'win_rate': win_rate,
                'rank': rank,
                'pick_rate': pick_rate,
                'ban_rate': ban_rate,
                'matches': matches
                })

In [122]:
champion_data_df = pd.DataFrame(champion_data)
champion_data_df #Data of all champions scraped from u.gg

,name,tier,win_rate,rank,pick_rate,ban_rate,matches
0,Aatrox,ATier,49.96%,64.41%,11.4%,16.2%,"286,783"
1,Ahri,CTier,49.66%,67.24%,8.9%,10.1%,"224,349"
2,Akali,DTier,49%,82.76%,4.8%,12.0%,"119,859"
3,Akshan,S+Tier,52.48%,5.17%,2.7%,8.6%,"67,908"
4,Alistar,STier,50.75%,27.27%,6.3%,2.3%,"158,238"
...,...,...,...,...,...,...,...
162,Zeri,CTier,49.72%,59.26%,12.0%,4.6%,"300,497"
163,Ziggs,DTier,47.88%,93.10%,1.4%,0.3%,"35,191"
164,Zilean,ATier,50.25%,40.91%,2.3%,0.7%,"57,974"
165,Zoe,ATier,50.89%,32.76%,2.5%,1.2%,"61,492"


<h2> Database Postgresql </h2>

<h4> Connection to DataBase for easy retrieval/insertion of data to prevent exceeding ratelimit on API and general efficient performance </h4>

In [5]:
from pangres import upsert
from sqlalchemy import text, create_engine

db_username = "postgres"
db_password = "123123123"
db_host = "localhost"
db_port = "5432"
db_name = "lol_analytics"

def create_db_connection_string(db_username, db_password, db_host, db_port, db_name):
        connection_url = 'postgresql+psycopg2://'+db_username+':'+db_password+'@'+db_host+':'+db_port+'/'+db_name
        return connection_url

conn_url = create_db_connection_string(db_username, db_password, db_host, db_port, db_name)

db_engine = create_engine(conn_url, pool_recycle=3600)

connection = db_engine.connect()

In [19]:
with db_engine.connect() as connection:
    leaderboard_df = pd.read_sql(text('SELECT * FROM "Top50SoloQ".leaderboard;'), connection)

leaderboard_df.head(300) #List of top 300 players in LOL

,summonerId,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,I_Ly9cVMnbqP3Lk_fuZbij0g57jAl1vM223hn0BY-Thlu14,2154,I,173,100,True,False,False,True
1,f-f6cnj0xVapOm-KtoQZ14gpHKZBVBu5TnajCeSPNcNna2M8,1661,I,199,137,True,False,False,False
2,ywLe0SCqv9aq3iPO_EWY9l_lnNa4_QtyMW0_w1csrlv5RhA,1551,I,188,136,True,False,False,True
3,xH7x54xRGwAXcxcwEa6mkANUFb2nYQuQcAEZuB7jOtadV1U,1481,I,140,86,True,False,False,False
4,cXAxF2EIp-hCfhvHDpQ6MFAb5TFHEE7aS5eFV8ZMNC_Spe...,1475,I,235,189,True,False,False,False
...,...,...,...,...,...,...,...,...,...
295,A9hwwGEslZDBJUbQ6DxtRuARy4agJvbhcamjSItlKKejMhk,723,I,99,49,False,False,True,False
296,ROXmKIkCunhuLDOemffR3hO-EPb5CKFK3jrUdRDr8BMHw5...,723,I,87,55,False,False,True,True
297,t4ho9gJ9qG9BUqKu7NpeCUCil4MLkkDDgzyb1VSqHonzDKM,723,I,143,120,True,False,False,False
298,UtrSNIsxG0DCLh3xJxPC8FARhI1t1JUPegs0G85I9jKeB1Y,721,I,119,74,False,False,True,True


In [20]:
with db_engine.connect() as connection:
    playerId_df = pd.read_sql(text('SELECT * FROM "playerId".summonerdata;'), connection)
playerId_df #list of top 90 players in LOL and their ids

,puuid,summonerId
0,Zl5uf2ERyvcZJD_4ELxzgc9SCaltAuDOjAIiuj2sdknOA1...,I_Ly9cVMnbqP3Lk_fuZbij0g57jAl1vM223hn0BY-Thlu14
1,lWMqB3Y5AuREUMTIq2uNUzYosZFcrsGm3UwKjuW8ZdQxQS...,f-f6cnj0xVapOm-KtoQZ14gpHKZBVBu5TnajCeSPNcNna2M8
2,iOI9wt8myzTiOUZpdqYumAQo1490Axk4sFvKp1ZTtz0fui...,xH7x54xRGwAXcxcwEa6mkANUFb2nYQuQcAEZuB7jOtadV1U
3,5XqBXMuSPFUrpk807hIZ5jJ-IGj2uyOgySWq6-GrXlDcmM...,ywLe0SCqv9aq3iPO_EWY9l_lnNa4_QtyMW0_w1csrlv5RhA
4,_7r_DQhl9ENq3mc9XYlpuESPeRyv3CS2lLLHrSN-WkrDvd...,cXAxF2EIp-hCfhvHDpQ6MFAb5TFHEE7aS5eFV8ZMNC_Spe...
...,...,...
85,gIkRkkSGeoDS4LK5eluixNDmvckztMfCm9AN17_IgCWGH0...,gglipYeO0beyFFmuXxDDBH5KHZE7jgSeLEnI5q4r5NqPrj...
86,IFEwVEBnESM6acu1JrkVgalzl2Bvfw-giEkbXmvwJ1IDJy...,xJEIxbDTUNxeZuj9ykXpCSAFoQcnJmwZoen5C_DjQYIvyA...
87,T10hxgwrcpB7dg9NiK0_UiNBSGa2YP9s8Xv1pYQW_hlgsq...,0K8ZCEQIPEKJkNLQMTQvkF8DrkXCuWmj_BeAp76aAIeBUd...
88,cLxnRRh2RfKTt3-XkIOHfl_ex8vNFI6dCbplwdIa3BjTaN...,aK3-PxDQ6nV8J3yj8OHF1e-lZJVCxhA_6xftXKxzCOT7lO...


<h2> Data Collection </h2>

<h4> API functions </h4>

In [273]:
api_key = "RGAPI-91201f36-f9c5-45b5-90a7-6c57c246fd34"

def get_puuid_fromSummonerId(summonerId):
    puuid = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/{summonerId}?api_key={api_key}").json()['puuid']
    return puuid

def getLeaderBoard(league):
    get_top50_url = f"https://na1.api.riotgames.com/lol/league/v4/{league}leagues/by-queue/RANKED_SOLO_5x5?api_key={api_key}"
    return requests.get(get_top50_url).json()

def get_matches(puuid):
    url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue=420&start=0&count=50&api_key={api_key}"
    return requests.get(url).json()

def get_matchData(matchId):
    url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key={api_key}"
    return requests.get(url).json()

<h4> Helper Functions </h4>

In [133]:
import time

def TopPlayers (): #function to get all players in the top 3 LOL leagues
    chal = getLeaderBoard('challenger')
    gm = getLeaderBoard('grandmaster')
    master = getLeaderBoard('master')

    chal_df = pd.DataFrame(chal['entries'])
    gm_df = pd.DataFrame(gm['entries'])
    master_df = pd.DataFrame(master['entries'])

    lb_df = pd.concat([chal_df, gm_df, master_df], ignore_index=True).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    lb_df = lb_df.set_index('summonerId')

    lb_df.to_sql('leaderboard', con=connection, schema='Top50SoloQ', if_exists='replace')

def PuuidWSummonerId(): #function to convert summonerId to Puuid and store it for easy conversions
    puuid_list = []
    count = 0
    for player in leaderboard_df.head(300)['summonerId']:
        if count == 99:
            count = 0
            time.sleep(90) #wait for ratelimit on api
        
        puuid_list.append({'puuid' : get_puuid_fromSummonerId(player),
                        'summonerId': player})
        count += 1

    puuid_df = pd.DataFrame(puuid_list)
    puuid_df.to_sql(name='summonerdata', con=connection, schema='playerId', if_exists='replace', index=True)

def processMatchInfo(match): #Cleans up individual Match data for dataframe that feeds to AI
    redTeamChamps = []
    redTeamRoles = []
    blueTeamChamps = []
    blueTeamRoles = []
    outcome = 0
    participants = match['metadata']['participants']
    players = match['info']['participants']

    # 2 indicates Red win, 0 indicates Blue win
    if match['info']['participants'][0]['win'] == False:
        if match['info']['participants'][0]['teamId'] == 200:
            outcome = 0
        else:
            outcome = 1
    else:
        if match['info']['participants'][0]['teamId'] == 200:
            outcome = 1
        else:
            outcome = 0

    def get_index(puuid):
        index = players[participants.index(puuid)]
        return index
    
    for player in participants:
        player = get_index(player)
        if player['teamId'] == 200:
            redTeamChamps.append(player['championId'])
            redTeamRoles.append(player['teamPosition'])
        else:
            blueTeamChamps.append(player['championId'])
            blueTeamRoles.append(player['teamPosition'])
    
    matchInfo = pd.DataFrame()

    for i in range(0,5):
        match = pd.DataFrame([{
            'outcome' : outcome,

            'redTeam' : redTeamChamps[i],
            'blueTeam' : blueTeamChamps[i],

            'Role' : redTeamRoles[i]
        }])
        matchInfo = pd.concat([matchInfo, match])

    return matchInfo

def merge_champion_stats(all_match_df, champion_data_df):
    all_matchData_df = all_match_df.copy()

    pattern = re.compile(r'^(redTeam|blueTeam)$')
    for index, row in all_match_df.iterrows():
        for col in all_match_df.columns:
            if pattern.match(col):
                champion_name = all_match_df.loc[index, col]
                stats = champion_data_df[champion_data_df['name'] == champion_name]
                if not stats.empty:
                    for stat_col in champion_data_df.columns:
                        if stat_col != 'name' and stat_col != 'matches' :
                            all_matchData_df.loc[index, f"{col}_{stat_col}"] = stats[stat_col].values[0]

    all_matchData_df[f'redTeam_win_rate'] = all_matchData_df[f'redTeam_win_rate'].str.rstrip('%').astype(float)
    all_matchData_df[f'redTeam_pick_rate'] = all_matchData_df[f'redTeam_pick_rate'].str.rstrip('%').astype(float)
    all_matchData_df[f'redTeam_ban_rate'] = all_matchData_df[f'redTeam_ban_rate'].str.rstrip('%').astype(float)
    all_matchData_df[f'redTeam_rank'] = all_matchData_df[f'redTeam_rank'].str.rstrip('%').astype(float)

    all_matchData_df[f'blueTeam_win_rate'] = all_matchData_df[f'blueTeam_win_rate'].str.rstrip('%').astype(float)
    all_matchData_df[f'blueTeam_pick_rate'] = all_matchData_df[f'blueTeam_pick_rate'].str.rstrip('%').astype(float)
    all_matchData_df[f'blueTeam_ban_rate'] = all_matchData_df[f'blueTeam_ban_rate'].str.rstrip('%').astype(float)
    all_matchData_df[f'blueTeam_rank'] = all_matchData_df[f'blueTeam_rank'].str.rstrip('%').astype(float)

    return all_matchData_df


#test_df = processMatchInfo(get_matchData('NA1_5035035400'))
#test_df = test_df.replace(championId_dict)
#test_df

<h4> Collection of list of matches played by top players </h4>

In [275]:
uniqueMatches = []
for player in playerId_df.head(100)['puuid']:
    for match in get_matches(player):
        if match not in uniqueMatches:
            uniqueMatches.append(match)

print(len(uniqueMatches))
uniqueMatches

2672


['NA1_5036518415',
 'NA1_5036511826',
 'NA1_5036496169',
 'NA1_5035936299',
 'NA1_5035925053',
 'NA1_5035906397',
 'NA1_5035470687',
 'NA1_5035428371',
 'NA1_5035402815',
 'NA1_5034824451',
 'NA1_5034811770',
 'NA1_5034780669',
 'NA1_5034768874',
 'NA1_5030494093',
 'NA1_5030442871',
 'NA1_5030412958',
 'NA1_5029179586',
 'NA1_5029163020',
 'NA1_5029143841',
 'NA1_5029123902',
 'NA1_5029104414',
 'NA1_5028495054',
 'NA1_5028023007',
 'NA1_5027852772',
 'NA1_5027255716',
 'NA1_5027250736',
 'NA1_5027232213',
 'NA1_5027220249',
 'NA1_5026029078',
 'NA1_5026012547',
 'NA1_5025970120',
 'NA1_5025953136',
 'NA1_5025933480',
 'NA1_5025327116',
 'NA1_5025296953',
 'NA1_5025267556',
 'NA1_5025240622',
 'NA1_5024892367',
 'NA1_5024850190',
 'NA1_5024299992',
 'NA1_5024088575',
 'NA1_5023446542',
 'NA1_5023313854',
 'NA1_5023286889',
 'NA1_5023261444',
 'NA1_5023248776',
 'NA1_5023228173',
 'NA1_5023207728',
 'NA1_5023196254',
 'NA1_5023183334',
 'NA1_5038448587',
 'NA1_5038432543',
 'NA1_503841

In [276]:
matchall = []
count = 0
totalC = 0
all_match_df = pd.DataFrame()
for match in uniqueMatches:
    count+=1
    totalC += 1
    print(totalC)
    if count == 90:
        count = 0
        time.sleep(100)
    matchall.append(processMatchInfo(get_matchData(match)))

all_match_df = pd.concat(matchall)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [135]:
all_match_df.reset_index(drop=True, inplace=True)
# Specify columns to replace
columns_to_replace = ['redTeam', 'blueTeam']

# Create a dictionary for specific columns
replacement_dict = {col: championId_dict for col in columns_to_replace}

# Replace values in specific columns
all_match_df.replace(replacement_dict, inplace=True)
#all_match_df = all_match_df.replace(championId_dict)

In [137]:
pd.options.display.max_columns = 100
matchup_df = merge_champion_stats(all_match_df, champion_data_df)

In [138]:
matchup_df

,outcome,redTeam,blueTeam,Role,redTeam_tier,redTeam_win_rate,redTeam_rank,redTeam_pick_rate,redTeam_ban_rate,blueTeam_tier,blueTeam_win_rate,blueTeam_rank,blueTeam_pick_rate,blueTeam_ban_rate
0,1,Kennen,Udyr,TOP,ATier,50.69,38.98,2.4,1.6,STier,51.88,5.88,3.4,2.3
1,1,Rengar,Nidalee,JUNGLE,DTier,48.61,88.24,2.7,5.4,S+Tier,50.17,50.98,9.2,12.7
2,1,Zed,Galio,MIDDLE,DTier,48.80,87.93,7.0,18.7,S+Tier,52.42,8.62,3.8,1.5
3,1,Kai'Sa,Ezreal,BOTTOM,S+Tier,50.80,29.63,33.7,12.4,DTier,49.56,74.07,25.2,15.1
4,1,Rakan,Seraphine,UTILITY,ATier,50.11,45.45,5.0,0.8,ATier,50.63,34.09,4.7,1.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,0,Sion,Urgot,TOP,DTier,48.08,88.14,2.3,0.4,ATier,51.16,13.56,2.3,0.5
826,0,Kindred,Kha'Zix,JUNGLE,STier,50.91,23.53,3.8,4.1,S+Tier,50.38,47.06,10.1,9.9
827,0,Orianna,LeBlanc,MIDDLE,DTier,48.98,84.48,3.6,0.6,STier,49.97,56.90,9.5,24.1
828,0,Ezreal,Draven,BOTTOM,DTier,49.56,74.07,25.2,15.1,S+Tier,50.09,40.74,5.7,25.6


<h2> AI Model Training/Testing </h2>

In [266]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

tiers = ['S+Tier', 'STier', 'ATier', 'BTier', 'CTier', 'DTier']

tier_label_encoder = LabelEncoder()

tier_label_encoder.fit(tiers)

tester_df = matchup_df.copy()

In [251]:
tester_df.isnull().sum()
nan_rows = tester_df.isnull().any()
nan_rows

outcome               False
redTeam               False
blueTeam              False
Role                  False
redTeam_tier          False
redTeam_win_rate      False
redTeam_rank          False
redTeam_pick_rate     False
redTeam_ban_rate      False
blueTeam_tier         False
blueTeam_win_rate     False
blueTeam_rank         False
blueTeam_pick_rate    False
blueTeam_ban_rate     False
dtype: bool

In [267]:
all_champions = []
for champ in championId_dict.values():
    all_champions.append(champ)
label_encoder = LabelEncoder()
label_encoder.fit(all_champions)

roles = ['TOP', 'JUNGLE', 'MIDDLE', 'BOTTOM', 'UTILITY']
onehot_encoder = OneHotEncoder(sparse_output=False)
onehot_encoder.fit([[role] for role in roles])

# Fit and transform the data
tester_df['redTeam'] = label_encoder.transform(tester_df['redTeam'])
tester_df['redTeam_tier'] = tier_label_encoder.transform(tester_df['redTeam_tier'])

tester_df['blueTeam'] = label_encoder.transform(tester_df['blueTeam'])
tester_df['blueTeam_tier'] = tier_label_encoder.transform(tester_df['blueTeam_tier'])

tester_df['Role'] = onehot_encoder.transform(tester_df[['Role']])

x = tester_df.head(760).drop(columns=['outcome', 'Role'])
y = tester_df.head(760)['outcome']

c:\Users\Ivanb\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(


In [268]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [269]:
# Initialize the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

In [270]:
# Train the model
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [271]:
# Make predictions
y_pred = rf_model.predict(X_test)

In [272]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.5526315789473685
